# Example Sample Hotel and Flight Booker Agent 

This solution will help you book flight tickets and hotel.  The scenario is a trip London Heathrow LHR Feb 20th 2024 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel.


# Azure AI एजेंट सेवा को इनिशियलाइज़ करें और **.env** से कॉन्फ़िगरेशन जानकारी प्राप्त करें

### **.env**

एक .env फाइल बनाएं

**.env** में Azure AI एजेंट सेवा का कनेक्शन स्ट्रिंग, AOAI द्वारा उपयोग किया जाने वाला मॉडल, और संबंधित Google API सर्च सेवा API, ENDPOINT, आदि होते हैं।

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "आपके Azure AI एजेंट सेवा मॉडल डिप्लॉयमेंट का नाम"

[**NOTE**] आपको 100,000 रेट लिमिट (टोकन्स प्रति मिनट) और 600 रेट लिमिट (रिक्वेस्ट प्रति मिनट) वाले मॉडल की आवश्यकता होगी।

  आप Microsoft Foundry - मॉडल और एंडपॉइंट में मॉडल प्राप्त कर सकते हैं।


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "आपके Azure AI एजेंट सेवा प्रोजेक्ट कनेक्शन स्ट्रिंग"

  आप AI Foundry पोर्टल स्क्रीन में अपने प्रोजेक्ट ओवरव्यू में प्रोजेक्ट कनेक्शन स्ट्रिंग प्राप्त कर सकते हैं।

- **SERPAPI_SEARCH_API_KEY** = "आपकी SERPAPI सर्च API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "आपका SERPAPI सर्च एंडपॉइंट"

Azure AI एजेंट सेवा के मॉडल डिप्लॉयमेंट नाम और प्रोजेक्ट कनेक्शन स्ट्रिंग प्राप्त करने के लिए, आपको Azure AI एजेंट सेवा बनानी होगी। इसे सीधे बनाने के लिए [इस टेम्पलेट](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) का उपयोग करने की सलाह दी जाती है (***ध्यान दें:*** Azure AI एजेंट सेवा वर्तमान में सीमित क्षेत्र में सेट है। क्षेत्र सेट करने के लिए [इस लिंक](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) को देखें)।

एजेंट को SERPAPI एक्सेस करना होगा। पंजीकरण के लिए [इस लिंक](https://serpapi.com/searches) का उपयोग करने की सलाह दी जाती है। पंजीकरण के बाद, आप एक अद्वितीय API KEY और ENDPOINT प्राप्त कर सकते हैं।


# सेटअप 

इस नोटबुक को चलाने के लिए, आपको यह सुनिश्चित करना होगा कि आपने आवश्यक लाइब्रेरीज़ को `pip install -r requirements.txt` रन करके इंस्टॉल किया है।


In [ ]:
from semantic_kernel import __version__

__version__

आपका सेमांटिक कर्नेल संस्करण कम से कम 1.27.2 होना चाहिए।


अपनी .env फाइल सेटिंग और संसाधन लोड करें कृपया सुनिश्चित करें कि आपने अपनी कुंजी और सेटिंग जोड़ी है और स्थानीय .env फाइल बनाई है।


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Azure में लॉग इन करें

अब आपको Azure में लॉग इन करना होगा। एक टर्मिनल खोलें और निम्नलिखित कमांड चलाएं:

```bash
az login
```

यह कमांड आपसे आपके Azure क्रेडेंशियल्स दर्ज करने के लिए कहेगा, जिससे Azure AI Agent सेवा सही तरीके से काम कर सकेगी।


# Explanation:
यह एक वेरिएबल है जो SERP (Search Engine Results Page) API सेवा तक पहुँचने के लिए API कुंजी संग्रहित करता है। एक API कुंजी एक अनूठा पहचानकर्ता है जिसका उपयोग आपके खाते से जुड़े अनुरोधों को प्रमाणित करने के लिए किया जाता है।

Purpose: इस पंक्ति का उद्देश्य API कुंजी को एक वेरिएबल में संग्रहित करना है ताकि इसे SERP API सेवा के अनुरोधों को प्रमाणित करने के लिए उपयोग किया जा सके। सेवा का उपयोग करने और खोज करने के लिए API कुंजी आवश्यक है।
How to Get a SERP API Key: SERP API कुंजी प्राप्त करने के लिए, https://serpapi.com पर निम्नलिखित सामान्य चरणों का पालन करें (विशिष्ट SERP API सेवा के अनुसार सटीक चरण भिन्न हो सकते हैं):

Choose a SERP API Service: कई SERP API सेवाएं उपलब्ध हैं, जैसे SerpAPI, Google Custom Search JSON API, और अन्य। अपनी आवश्यकताओं के अनुसार सबसे उपयुक्त सेवा चुनें।

Sign Up for an Account: चुनी हुई SERP API सेवा की वेबसाइट पर जाएं और एक खाता बनाएं। आपको कुछ बुनियादी जानकारी प्रदान करनी पड़ सकती है और अपना ईमेल पता सत्यापित करना पड़ सकता है।

Create an API Key: साइन अप करने के बाद, अपने खाते में लॉग इन करें और API सेक्शन या डैशबोर्ड पर जाएं। एक नई API कुंजी बनाने या उत्पन्न करने का विकल्प देखें।
Copy the API Key to your .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: यह एक वेरिएबल है जो SERP API एंडपॉइंट के लिए बेस URL को संग्रहीत करता है। वेरिएबल का नाम BASE_URL एक कन्वेंशन है जो यह दर्शाता है कि यह URL API अनुरोध करने के लिए शुरुआती बिंदु है।
'https://serpapi.com/search':

यह वास्तविक URL स्ट्रिंग है जो BASE_URL वेरिएबल को असाइन की गई है। यह SERP API का उपयोग करके खोज क्वेरीज करने के लिए एंडपॉइंट को दर्शाता है।

# Purpose:
इस लाइन का उद्देश्य एक स्थिरांक को परिभाषित करना है जो SERP API के लिए बेस URL को रखता है। इस URL का उपयोग API अनुरोध बनाने के लिए शुरुआती बिंदु के रूप में किया जाएगा ताकि खोज संचालन किया जा सके।

# Usage:
बेस URL को वेरिएबल में परिभाषित करके, आप इसे अपने कोड में कहीं भी आसानी से पुन: उपयोग कर सकते हैं जब भी आपको SERP API को अनुरोध करने की आवश्यकता होती है। यह आपके कोड को अधिक रख-रखाव योग्य बनाता है और कई जगहों पर हार्डकोडिंग से संबंधित त्रुटियों का जोखिम कम करता है। वर्तमान उदाहरण https://serpapi.com/search?engine=bing है जो Bing सर्च API का उपयोग कर रहा है। अलग-अलग API को https://Serpapi.com पर चुना जा सकता है।


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# व्याख्या:

यहाँ आपका प्लगइन कोड स्थित है।

क्लास परिभाषा: `class BookingPlugin`: एक क्लास है जिसका नाम BookingPlugin है और इसमें होटल और फ्लाइट बुकिंग के लिए मेथड्स शामिल हैं।

होटल बुकिंग मेथड:

- `@kernel_function(description="booking hotel")`: एक डेकोरेटर जो इस फंक्शन को होटल बुकिंग के लिए कर्नेल फंक्शन के रूप में वर्णित करता है।
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: होटल बुकिंग के लिए एक मेथड परिभाषित करता है जिसमें एनोटेटेड पैरामीटर और रिटर्न टाइप होते हैं।

यह मेथड होटल बुकिंग अनुरोध के लिए पैरामीटर का एक डिक्शनरी बनाता है और SERP API को GET अनुरोध भेजता है। यह प्रतिक्रिया की स्थिति की जाँच करता है और यदि सफल होती है तो होटल की संपत्तियों को वापस करता है, अन्यथा None लौटाता है।

फ्लाइट बुकिंग मेथड:

- `@kernel_function(description="booking flight")`: एक डेकोरेटर जो इस फंक्शन को फ्लाइट बुकिंग के लिए कर्नेल फंक्शन के रूप में वर्णित करता है।
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: फ्लाइट बुकिंग के लिए एक मेथड परिभाषित करता है जिसमें एनोटेटेड पैरामीटर और रिटर्न टाइप होते हैं।

यह मेथड आउटकॉन्ड और रिटर्न फ्लाइट अनुरोधों के लिए पैरामीटर के डिक्शनरी बनाता है और SERP API को GET अनुरोध भेजता है। यह प्रतिक्रिया की स्थिति की जाँच करता है और यदि सफल होती है तो फ्लाइट जानकारी को परिणाम स्ट्रिंग में जोड़ता है, अन्यथा एक त्रुटि संदेश मुद्रित करता है। मेथड फ्लाइट जानकारी वाली परिणाम स्ट्रिंग लौटाता है।


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# व्याख्या:
इंपोर्ट स्टेटमेंट्स: Azure प्रमाण पत्र, AI एजेंट, चैट संदेश सामग्री, लेखक भूमिका, और कर्नेल फ़ंक्शन डेकोरेटर के लिए आवश्यक मॉड्यूल आयात करें।

एसिंक्रोनस कॉन्टेक्स्ट मैनेजर: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): यह Azure प्रमाण पत्र को संभालने और AI एजेंट क्लाइंट बनाने के लिए एक एसिंक्रोनस कॉन्टेक्स्ट मैनेजर सेट करता है।

एजेंट का नाम और निर्देश:
- `AGENT_NAME = "BookingAgent"`: एजेंट का नाम परिभाषित करता है।
- `AGENT_INSTRUCTIONS = """..."""`: एजेंट को बुकिंग अनुरोधों को कैसे संभालना है, इसके लिए विस्तृत निर्देश देता है।

एजेंट परिभाषा बनाएँ: `agent_definition = await client.agents.create_agent(...)`: निर्दिष्ट मॉडल, नाम, और निर्देशों के साथ एक एजेंट परिभाषा बनाता है।

AzureAI एजेंट बनाएँ: `agent = AzureAIAgent(...)`: क्लाइंट, एजेंट परिभाषा, और परिभाषित प्लगइन का उपयोग करके एक AzureAI एजेंट बनाता है।

थ्रेड बनाएँ: `thread: AzureAIAgentThread | None = None`: एजेंट के लिए एक थ्रेड बनाएँ। पहले थ्रेड बनाना आवश्यक नहीं है - यदि `None` मान दिया जाता है, तो पहली कॉल के दौरान एक नया थ्रेड बनाया जाएगा और उत्तर के हिस्से के रूप में लौटाया जाएगा।

उपयोगकर्ता इनपुट: `user_inputs = ["..."]`: एजेंट द्वारा संसाधित करने के लिए उपयोगकर्ता इनपुट की एक सूची परिभाषित करता है।

अंत में ब्लॉक में, संसाधनों को साफ़ करने के लिए थ्रेड और एजेंट को हटा दें।


# प्रमाणीकरण

`DefaultAzureCredential` क्लास Azure SDK for Python का हिस्सा है। यह Azure सेवाओं के साथ प्रमाणीकरण करने का एक डिफ़ॉल्ट तरीका प्रदान करता है। यह एक विशिष्ट क्रम में कई तरीकों का उपयोग करके प्रमाणीकरण करने का प्रयास करता है, जैसे कि पर्यावरण चर, प्रबंधित पहचान, और Azure CLI क्रेडेंशियल्स।

असिंक संचालन: aio मॉड्यूल यह दर्शाता है कि DefaultAzureCredential क्लास असिंक संचालन का समर्थन करता है। इसका मतलब है कि आप इसे asyncio के साथ उपयोग कर सकते हैं ताकि नॉन-ब्लॉकिंग प्रमाणीकरण अनुरोध किए जा सकें।


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
यह दस्तावेज़ एआई अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनूदित किया गया है। यद्यपि हम सटीकता के लिए प्रयासरत हैं, कृपया यह ध्यान रखें कि स्वचालित अनुवाद में त्रुटियाँ या अशुद्धियाँ हो सकती हैं। मूल दस्तावेज़ अपनी मूल भाषा में ही अधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सलाह दी जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
